In [ ]:
import socket
import time
import random
import threading
from datetime import datetime

def simulate_data():
    # 產生 7 個隨機浮點數，並組成預期格式的數據字串
    data_values = [str(round(random.uniform(0, 10), 2)) for _ in range(7)]
    return "data_5610:" + ",".join(data_values) + "\n"

def send_normal_data(client_socket):
    """持續發送模擬數據"""
    while True:
        data = simulate_data()
        try:
            client_socket.sendall(data.encode())
            print("Sent:", data.strip())
        except Exception as e:
            print("Error sending normal data:", e)
            break
        time.sleep(0.04)  # 調整數據發送頻率

def send_ready_messages(client_socket):
    """定時發送 'ready to predict' 訊息"""
    while True:
        try:
            # 依序發送右腳與左腳的預測訊息
            ready_right = "ready to predict right leg\n"
            client_socket.sendall(ready_right.encode())
            print("Sent:", ready_right.strip())
            time.sleep(1.5)  # 每隔 5 秒發送一次右腳訊息
            
            ready_left = "ready to predict left leg\n"
            client_socket.sendall(ready_left.encode())
            print("Sent:", ready_left.strip())
            time.sleep(1.7)  # 每隔 5 秒發送一次左腳訊息
        except Exception as e:
            print("Error sending ready messages:", e)
            break

def main():
    host = 'localhost'
    port = 23456  # 與接收端設定的埠號需一致
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect((host, port))
    
    # 啟動發送模擬數據的線程
    thread_normal = threading.Thread(target=send_normal_data, args=(client_socket,))
    thread_normal.daemon = True
    thread_normal.start()
    
    # 啟動定時發送 ready message 的線程
    thread_ready = threading.Thread(target=send_ready_messages, args=(client_socket,))
    thread_ready.daemon = True
    thread_ready.start()
    
    try:
        # 主線程持續運行，防止程式退出
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("發送中斷，關閉 socket")
        client_socket.close()

if __name__ == '__main__':
    main()


In [1]:
import socket
import time
import threading
import pandas as pd
from datetime import datetime


def read_data():
    # 讀取 IMU 數據檔案
    imu_data = pd.read_csv('/home/ndt/Desktop/predict_model/test_IMU_data/2/20241008-2HSFoot.csv', header=1)
    # 讀取特定時間檔案
    specific_time = pd.read_csv('/home/ndt/Desktop/predict_model/test_IMU_data/2/20241008-2HShstime.csv')

    # 確認資料的欄位名稱
    print("IMU Data Columns:", imu_data.columns)

    # 將 'time' 欄位轉換為浮點數
    imu_data['time'] = pd.to_numeric(imu_data['time'], errors='coerce')
    # 提取右腳和左腳的特定時間列表
    right_HS_times = specific_time['rightfoot'].values
    left_HS_times = specific_time['leftfoot'].values

    return imu_data, right_HS_times, left_HS_times

def send_data(client_socket, imu_data, right_HS_times, left_HS_times):
    index = 0
    previous_time = 0.0  # 用來記錄前一個時間點

    while index < len(imu_data):
        row = imu_data.iloc[index]

        # 取得時間與右腳 IMU 數據
        current_time = row['time']
        data_values = [
            str(row['ax']), str(row['ay']), str(row['az']),
            str(row['wx']), str(row['wy']), str(row['wz'])
        ]
        data_string = f"data_5610:{current_time},{','.join(data_values)}\n"

        try:
            # 發送 IMU 數據
            client_socket.sendall(data_string.encode())
            print(f"{datetime.now()}""Sent:", data_string.strip())

            # 檢查是否要發送 ready message
            if current_time in right_HS_times:
                ready_right = "ready to predict left leg\n"
                client_socket.sendall(ready_right.encode())
                print("Sent:", ready_right.strip())

            if current_time in left_HS_times:
                ready_left = "ready to predict right leg\n"
                client_socket.sendall(ready_left.encode())
                print("Sent:", ready_left.strip())

        except Exception as e:
            print("Error sending data:", e)
            break

        # 計算當前時間點與前一個時間點的差值 (秒)
        if index > 0:
            time_difference = current_time - previous_time
            time.sleep(time_difference)  # 依照實際的時間差來延遲

        previous_time = current_time
        index += 1

    # 所有數據發送完畢，發送 END 訊息
    try:
        end_message = "END\n"
        client_socket.sendall(end_message.encode())
        print("Sent:", end_message.strip())
    except Exception as e:
        print("Error sending END message:", e)

def main():
    host = 'localhost'
    port = 34567  # 與接收端設定的埠號需一致
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect((host, port))

    imu_data, right_HS_times, left_HS_times = read_data()

    try:
        send_data(client_socket, imu_data, right_HS_times, left_HS_times)
    except KeyboardInterrupt:
        print("發送中斷，關閉 socket")
    finally:
        client_socket.close()
if __name__ == '__main__':
    main()


IMU Data Columns: Index(['time', 'ax', 'ay', 'az', 'wx', 'wy', 'wz', 'ax.1', 'ay.1', 'az.1',
       'wx.1', 'wy.1', 'wz.1', 'pelvic_rotation'],
      dtype='object')
2025-03-18 14:13:18.764853Sent: data_5610:0.0,9.118929862976074,-2.5879695415496826,0.2807702720165252,-0.1014740392565727,0.4774620831012726,0.4454176425933838
2025-03-18 14:13:18.765217Sent: data_5610:0.0901976,9.6804048438605,-2.111958901646896,0.6835646132920564,0.0074732068178224,0.4603736169105087,0.410171774686962
2025-03-18 14:13:18.856071Sent: data_5610:0.1409496999999999,9.82693461158525,-1.7581160167840324,1.70931146016091,-0.0962164288356425,-0.1323912667565197,-0.3876895382151658
2025-03-18 14:13:18.907495Sent: data_5610:0.2129664,9.741507530212402,-0.6994218755802851,0.6591997742652893,0.1341827691352796,-0.2644624867772501,-0.4353318245160996
2025-03-18 14:13:18.980149Sent: data_5610:0.2697335,10.024530948139011,-0.404651828226544,0.5375709643838715,0.0156256604012667,-0.2174666414644244,-0.3702896926633104
